In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import os
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
from sklearn.metrics import r2_score
from scipy.interpolate import interp1d
import tensorflow_datasets as tfds
from collections import Counter
import scipy.ndimage
from tensorflow.keras.callbacks import ModelCheckpoint
import keras
import gc
import random
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import tarfile
import urllib.request
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.nn.functional as F
from scipy.stats import gaussian_kde
from numba import jit
from torchvision.datasets import Places365
from tqdm import tqdm  # 導入進度條庫
import json
import torch.nn as nn

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
torch.backends.cudnn.benchmark = True
limit = 1000

dropout_rate = 0.0
num_samples = 1  # MC Dropout樣本數量

# bzq modifying
len_x_target = 20
len_y_target = 20
stride_x_target = 10
stride_y_target = 10

# mean, std proportion
alpha = 0.5

bins_size = 30  # 統計採樣數
poly_degree = bins_size - 1
window_size = 1

#target image preprocessing
angle = 0
pixels = 0

In [ ]:
# 設置設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),  # Convert to tensor (values in range 0–1)
    transforms.Lambda(lambda x: x * 255.0),  # Scale to range 0–255
    transforms.Lambda(lambda x: x[[2, 1, 0], ...]),  # Convert RGB to BGR
])

# 建立隨機影像數據集（不依賴 Places365）
class RandomImageDataset(torch.utils.data.Dataset):
    def __init__(self, num_images=1000, image_size=(224, 224)):
        self.num_images = num_images
        self.image_size = image_size

    def __len__(self):
        return self.num_images

    def __getitem__(self, idx):
        random_image = np.random.randint(0, 256, (self.image_size[1], self.image_size[0], 3), dtype=np.uint8)
        image = Image.fromarray(random_image)
        return transform(image)

# 使用自定義隨機影像數據集
dataset = RandomImageDataset(num_images=limit)

In [ ]:
# 初始化 VGG16，無預訓練權重
model = models.vgg16(weights=False)

# 修改 classifier 最後一層，適應 365 類別
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 365)

 

# 為 features 添加 Dropout
new_features = []
for layer in model.features:
    new_features.append(layer)
    if isinstance(layer, nn.ReLU):  # 在 ReLU 之後插入 Dropout
        new_features.append(nn.Dropout(p=dropout_rate))

model.features = nn.Sequential(*new_features)

# 定義新的 classifier，刪除第3個和第7個 Dropout
new_classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.1, inplace=False),  # 保留 p=0.1
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.1, inplace=False),  # 保留 p=0.1
    nn.Linear(4096, 365)
)

# 替換原本的 classifier
model.classifier = new_classifier

# 加載保存的權重
if os.path.exists("vgg16_places365.pt"):
    checkpoint = torch.load("vgg16_places365.pt")
    
    # 映射權重 (Caffe -> PyTorch)
    mapped_state_dict = {  
        "features.0.weight": checkpoint["conv1_1.weight"],
        "features.0.bias": checkpoint["conv1_1.bias"],
        "features.3.weight": checkpoint["conv1_2.weight"],
        "features.3.bias": checkpoint["conv1_2.bias"],
        "features.7.weight": checkpoint["conv2_1.weight"],
        "features.7.bias": checkpoint["conv2_1.bias"],
        "features.10.weight": checkpoint["conv2_2.weight"],
        "features.10.bias": checkpoint["conv2_2.bias"],
        "features.14.weight": checkpoint["conv3_1.weight"],
        "features.14.bias": checkpoint["conv3_1.bias"],
        "features.17.weight": checkpoint["conv3_2.weight"],
        "features.17.bias": checkpoint["conv3_2.bias"],
        "features.20.weight": checkpoint["conv3_3.weight"],
        "features.20.bias": checkpoint["conv3_3.bias"],
        "features.24.weight": checkpoint["conv4_1.weight"],
        "features.24.bias": checkpoint["conv4_1.bias"],
        "features.27.weight": checkpoint["conv4_2.weight"],
        "features.27.bias": checkpoint["conv4_2.bias"],
        "features.30.weight": checkpoint["conv4_3.weight"],
        "features.30.bias": checkpoint["conv4_3.bias"],
        "features.34.weight": checkpoint["conv5_1.weight"],
        "features.34.bias": checkpoint["conv5_1.bias"],
        "features.37.weight": checkpoint["conv5_2.weight"],
        "features.37.bias": checkpoint["conv5_2.bias"],
        "features.40.weight": checkpoint["conv5_3.weight"],
        "features.40.bias": checkpoint["conv5_3.bias"],
        "classifier.0.weight": checkpoint["fc6.weight"],
        "classifier.0.bias": checkpoint["fc6.bias"],
        "classifier.3.weight": checkpoint["fc7.weight"],
        "classifier.3.bias": checkpoint["fc7.bias"],
        "classifier.6.weight": checkpoint["fc8a.weight"],
        "classifier.6.bias": checkpoint["fc8a.bias"],
    }
    #print(model.features)

    model.load_state_dict(mapped_state_dict, strict=False)

# **啟用 MC Dropout**
def enable_dropout(m):
    if isinstance(m, nn.Dropout):
        m.train()


model = model.to(device)
if dropout_rate > 0.0:
    print("enable dropout!")
    model.apply(enable_dropout)  # 讓 Dropout 在推論時啟用
model.eval()  # 設定評估模式，但 Dropout 仍作用


In [ ]:
# 得到了bzq 正確的函數，拿來做imagenet 正確的預測

def single_data_bzq_mask_preprocessing_imagenet(original_data, start_x, start_y, len_x, len_y, magnification):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data)
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] *= magnification
    #new_data[start_y:start_y + len_y, start_x:start_x + len_x, :] *= magnification
    return new_data


#print(random_num_for_bzq_mask_imagenet)

def single_data_bzq_mask_preprocessing_imagenet_random_global(original_data, start_x, start_y, len_x, len_y, random_num_for_bzq_mask_imagenet):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data)
    random_num_for_bzq_mask_imagenet = random_num_for_bzq_mask_imagenet[:, :len_y, :len_x] 
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] = random_num_for_bzq_mask_imagenet
    return new_data


bzq = []
correct_predictions_imagenet = []
incorrect_predictions_imagenet = []
bzq_imagenet = []

#bzq = 0的時候，提取mask之下的softmax
correct_predictions_bzq_zero_softmax_mean = []
correct_predictions_bzq_zero_softmax_std = []
incorrect_predictions_bzq_zero_softmax_mean = []
incorrect_predictions_bzq_zero_softmax_std = []


In [ ]:
##################################################################################################################################
def reverse_normalize(image, mean, std):
    # 確保均值和標準差是 numpy array，以支持逐像素計算
    mean = np.array(mean).reshape(1, 1, -1)  # [1, 1, C]
    std = np.array(std).reshape(1, 1, -1)    # [1, 1, C]

    # 反標準化公式
    image = (image * std) + mean
    return np.clip(image, 0, 255)  # 確保範圍在 [0, 255]

def normalize(image, mean, std):
    # 將影像標準化
    mean = np.array(mean).reshape(1, 1, -1)  # [1, 1, C]
    std = np.array(std).reshape(1, 1, -1)    # [1, 1, C]
    image = (image - mean) / std
    return image
    
test_dataset = dataset
image = test_dataset[0]
print(image.shape)

# 創建 DataLoader
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=6, pin_memory=True)

In [ ]:

img = test_dataset[0]

print(img)

#img = reverse_normalize(image.numpy().transpose(1, 2, 0), mean=[103.939, 116.779, 123.68], std=[1.0, 1.0, 1.0])
# 顯示影像
plt.imshow(image.numpy().transpose(1, 2, 0) / 255.0)
plt.axis("off")  # 隱藏座標軸
plt.title("Sample Image")  # 標題，可選
plt.show()

In [ ]:
# 預測並顯示分數
conf_vanilla = []

correct = 0
total = 0

conf_dropout = []  # 用於儲存最大置信度（Softmax）
predictions = []  # 用於儲存每次推理的預測結果
entropies = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device).float()
        
        batch_predictions = []
        batch_conf = []
        batch_entropy = []
        for _ in range(num_samples):
            outputs = model(images)  # 模型推理
            softmax_probs = torch.nn.functional.softmax(outputs, dim=1)
            
            max_probs, pred = torch.max(softmax_probs, dim=1)  # 最大置信度和預測
            batch_conf.extend(max_probs.cpu().numpy())  # 添加置信度
            batch_predictions.append(pred.cpu().numpy())
            # 計算熵
            entropy = (-softmax_probs * torch.log(softmax_probs)).sum(dim=1)
            batch_entropy.extend(entropy.cpu().numpy())
        conf_dropout.append(np.mean(batch_conf))
        predictions.append(np.array(batch_predictions))
        entropies.append(np.mean(batch_entropy))  # 記錄平均熵
        
predictions = np.array(predictions)  # 將所有預測結果轉換為陣列

In [ ]:
#print(entropies)

# 計算熵的分佈（分桶）
bins = np.linspace(min(entropies), max(entropies), 15) 

hist, bin_edges = np.histogram(entropies, bins=bins)  # 計算直方圖數據
print(bin_edges[:-1], hist)
# 轉換為折線圖
plt.figure(figsize=(10, 5))
plt.plot(bin_edges[:-1], hist, linestyle='-', marker='o')

# 設定標題與軸標籤
plt.title("Entropy")
plt.xlabel("Entropy")
plt.ylabel("Frequency")
plt.xlim(0.6, 1.8)
plt.legend()
plt.grid(True)
plt.show()